## User input data guideline for 18 features

From 01. Gender : [Male : 0, Femail: 1]

To 18. Occupation_type: [ ]

In [1]:
import os
import piheaan as heaan
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

## Create an input window to enter 18 variables

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QScrollArea

class InputWindow(QWidget):
    def __init__(self, questions):
        super().__init__()
        self.questions = questions
        self.user_input = []
        self.user_input_value = []
        self.initUI()

    def initUI(self):
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        content_widget = QWidget()
        layout = QVBoxLayout(content_widget)

        for title, prompt in self.questions:
            label = QLabel(f"{title}\n{prompt}")
            layout.addWidget(label)
            line_edit = QLineEdit()
            layout.addWidget(line_edit)
            self.user_input.append(line_edit)

        submit_button = QPushButton("Submit")
        submit_button.clicked.connect(self.onSubmit)
        layout.addWidget(submit_button)

        scroll_area.setWidget(content_widget)
        main_layout = QVBoxLayout(self)
        main_layout.addWidget(scroll_area)
        self.setLayout(main_layout)
        self.setWindowTitle("User Input")
        self.resize(400, 600)  # 창의 크기 조정
        self.show()

    def onSubmit(self):
        user_input = []
        for line_edit in self.user_input:
            value = line_edit.text()
            user_input.append(float(value))
        self.user_input_value = user_input
        self.close()

if __name__ == "__main__":
    questions = [
        ("What is your Gender?", "[Male : 0, Female : 1]"),
        ("Do you have your own car?", "[No : 0, Yes : 1]"),
        ("Do you have your own property?", "[No : 0, Yes : 1]"),
        ("Do you have your own phone for workplace?", "[No : 0, Yes : 1]"),
        ("Do you have your own phone for dailylife?", "[No : 0, Yes : 1]"),
        ("Do you have your own E-mail?", "[No : 0, Yes : 1]"),
        ("Do you have your own job?", "[No : 0, Yes : 1]"),
        ("How many children do you have?", "[Number of Children : 0, 1, 2...]"),
        ("How many family members do you have?", "[Number of Family member : 0, 1, 2...]"),
        ("When did you start to make transaction in this account?", "[Length of transaction period : 0,1,2...]"),
        ("What is amount of your total income?", "[Amount of total income : 1000,2000,3000...]"),
        ("What is your age?", "[Your own age : 20,25,30...]"),
        ("How long have you been employted?", "[Year that you are employed : 10, 15, 20...]"),
        ("What is your income type?", "['Commercial associate': 0, 'Pensioner': 1, 'State servant': 2, 'Student': 3, 'Working': 4]"),
        ("What is your educational type?", "['Academic degree': 0, 'Higher education': 1, 'Incomplete higher': 2, 'Lower secondary': 3, 'Secondary / secondary special': 4]"),
        ("What is your family status?", "['Civil marriage': 0, 'Married': 1, 'Separated': 2, 'Single / not married': 3, 'Widow': 4]"),
        ("What is your housing type?", "['Co-op apartment': 0, 'House / apartment': 1, 'Municipal apartment': 2, 'Office apartment': 3, 'Rented apartment': 4, 'With parents': 5]"),
        ("What is your occupation type?", "['Accountants': 0, 'Cleaning staff': 1, 'Cooking staff': 2, 'Core staff': 3, 'Drivers': 4, 'HR staff': 5, 'High skill tech staff': 6, 'IT staff': 7, 'Laborers': 8, 'Low-skill Laborers': 9, 'Managers': 10, 'Medicine staff': 11, 'Other': 12, 'Private service staff': 13, 'Realty agents': 14, 'Sales staff': 15, 'Secretaries': 16, 'Security staff': 17, 'Waiters/barmen staff': 18]")
    ]

    app = QApplication(sys.argv)
    window = InputWindow(questions)
    sys.exit(app.exec_())

## User input data encryption

In [4]:
# Create a key using the key generator
params = heaan.ParameterPreset.FGb
context = heaan.make_context(params)
heaan.make_bootstrappable(context)

KEY_FILE_PATH = "./keys"

sk = heaan.SecretKey(context)
os.makedirs(KEY_FILE_PATH, mode=0x1ed, exist_ok=True)
sk.save(KEY_FILE_PATH + "/secretkey.bin")

key_generator = heaan.KeyGenerator(context, sk)
key_generator.gen_common_keys()
key_generator.save(KEY_FILE_PATH + "/")

# Key Load
pk = heaan.KeyPack(context, KEY_FILE_PATH + "/")
pk.load_enc_key()
pk.load_mult_key()

enc = heaan.Encryptor(context)

LOG_SLOTS = 15  
NUM_SLOTS = 2**LOG_SLOTS

# Encrypts user-entered data and stores it in a vector in user_data_ctx
msg = heaan.Message(LOG_SLOTS)

for i in range(len(window.user_input_value)):
    msg[i] = window.user_input_value[i]

user_data_ctx = heaan.Ciphertext(context)
enc.encrypt(msg, pk, user_data_ctx)

## get its own Feature_importance and encrypt it

- Get Feature_importance, but there are about 10,000 in training_data and test_data, excluding ID and target columns, so get its own Feature_importance and encrypt it

In [5]:
# Load the data
X_data = pd.read_csv('X_data.csv')
y_data = pd.read_csv('y_data.csv')

# Learn RF models with plaintext data and obtain feature_importance
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_data, y_data)

# Weighted value encryption
weights_msg = heaan.Message(LOG_SLOTS)
for i, value in enumerate(rf.feature_importances_):
    weights_msg[i] = float(value)

weights_ctx = heaan.Ciphertext(context)
enc.encrypt(weights_msg, pk, weights_ctx)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\TempFolder\ipykernel_8756\2478523036.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_data, y_data)


## Find and decrypt a new customer's financial score by performing weight and input_data isomorphic cryptography operations

In [10]:
# Calculate financial scores with isomorphic computation
eval = heaan.HomEvaluator(context, pk)
dec = heaan.Decryptor(context)

score_ctx = heaan.Ciphertext(context) # Create empty ciphertext
result = heaan.Ciphertext(context)

# Multiply the first attribute by the weight and store it in score_ctx
# Multiply the remaining attributes and weights, and add them to the score_ctx
tmp = heaan.Ciphertext(context)
eval.mult(user_data_ctx, weights_ctx, tmp)
eval.add(score_ctx, tmp, score_ctx)
eval.left_rotate_reduce(score_ctx, 1, NUM_SLOTS, result)

# Decrypts stored financial scores as a result of the operation
score_msg = heaan.Message(LOG_SLOTS)
dec.decrypt(result, sk, score_msg)

# print("Financial Score:", score_msg) - All arrays to be printed
print("Financial Score:", score_msg[0])

Financial Score: (323.57912837712286+0j)
